In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = True

import matplotlib as mpl
mpl.rc("figure", dpi=200)

from ubergauss.tools import xmap
import numpy as np

# we have 3 modules:

# load 

In [ ]:
from cellsaw import load as loader
labels = loader.get41names()[7:16]
shortlabels = [label[:label.index('_')] for label in labels]
loadparams = {'path' : '/home/ubuntu/repos/natto/natto/data',
              'remove_unlabeled':True, 'mingenes': 200, 
              'preprocessingmethod':'natto', "subsample":1000}

def load(names):
    # load 100 adata will have:
    # "true" cluster annotation
    # 'name' dataset name 
    adatas = [loader.easyLoad100(x,**loadparams) for x in names]
    # this will add the true celltype labels for the pangalodb
    # 'truecelltyp'
    return loader.loadadata.annotatepangalo(loadparams['path'], adatas, names)


#adatas = xmap(lambda x:load.easyLoad100(x,**loadparams), labels)

# similarity
### dendro

In [ ]:
from cellsaw import similarity 
# test if mp is necessary, jaccard is missing, import others from nattp/optimize/util

sim = lambda a,b,c: similarity.cosine(*load([a,b]))
distances = similarity.matrixmap_mp(sim,labels,repeats = 2)
distances = np.mean(distances, axis =2)

similarity.draw.dendro(distances, shortlabels, distancecut =1)

### effective ranges

In [ ]:
from cellsaw import similarity 


jug = list(range(100,2000,200))

def calc(n_genes):
    distances = similarity.matrixmap_mp(sim,labels,repeats = 2)    
    valz = [[similarity.precision(distances[:,:,i], shortlabels,k), i,k,n_genes] 
                  for i in range(distances.shape[2]) 
                      for k in [1,2,3] ]
    # [val,rep,p@k,n_genes]
    return valz


values = [ i  for x in jug for i in calc(x)]
similarity.draw.plotPrecision(values)

# merge

In [ ]:
m = similarity.matrixmap_mp(sim,labels,repeats = 2)
m= np.mean(m, axis  =2)
neighs = similarity.getNeighbors(m,labels,k=1)
print(neighs)

In [ ]:
from cellsaw import merge
pairs =xmap(load,neighs)

mergedPairs = xmap(lambda x:merge.merge(x,umaps=[10,2]), pairs) 


labels = xmap(lambda x: x.getlabels(masked=[1]), mergedPairs)

labels = xmap(lambda x:merge.diffuse(*x), zip(mergedPairs,labels)) 



In [ ]:
import lmz
from cellsaw.merge import hungarianEM
for mpair in mergedPairs:
    
    # plot original labels
    labels = mpair.getlabels(masked=[])
    mpair.plot(labels)
    
    # plot distance hungarian
    #mpair.plot([mpair.hungdist[0]]*2, grad=True)
    
    # plot diffused labels
    labels = mpair.getlabels(masked=[1])
    dlabels = merge.diffuse(mpair, labels, pid =2, neighbors = 7) 
    mpair.plot(dlabels)
    
    print("#"*80)
    # hung and diffuse
    continue
    l2,e2,_,_ = hungarianEM.HEM(mpair,pid = 3)
    l2[e2] =-1
    mpair.plot([l2,l2])
    mpair.confuse2(l)

In [ ]:
import cellsaw.merge.eval as eva
eva.evaluate(mergedPairs, labels, setid = 1)